In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

print(tf.__version__)

2.15.0


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# loading dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [5]:
input_shape = (28, 28, 1)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train = x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test = x_test / 255.0

In [6]:
# one hot encoding
# y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [7]:
record_defaults = [tf.constant([], dtype=tf.float32)] * 10

y_train = tf.data.experimental.CsvDataset(
    filenames=["/content/drive/MyDrive/noisy_y_train.csv"], record_defaults=record_defaults
)

In [8]:
# Convert the CsvDataset to a list of EagerTensors
y_train = [tf.convert_to_tensor(value) for value in y_train]

In [9]:
len(y_train)

60000

Getting model and all original hyperparameters

In [10]:
batch_size = 64
num_classes = 10
num_epochs = 30

In [11]:
optimizer = tf.keras.optimizers.SGD(name="SGD")
cce = tf.keras.losses.CategoricalCrossentropy()

In [12]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(32,(5, 5), padding="same", activation="relu", input_shape=input_shape, name="conv1_1"),
        tf.keras.layers.Conv2D(32, (5, 5), padding="same", activation="relu", name="conv1_2"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu", name="conv2_1"),
        tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu", name="conv2_2"),
        tf.keras.layers.MaxPool2D(strides=(2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu", name="FC_1"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation="relu", name="FC_2"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax", name="FC_softmax"),
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.SGD(name="SGD"),
    loss="categorical_crossentropy",
    metrics=["acc"],
)

In [13]:
# Load gradients from the checkpoint file
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint.restore("/content/drive/MyDrive/epoch_layer_gradients.ckpt-1").expect_partial()

In [14]:
# get indices with noisy data
indices = (np.loadtxt("/content/drive/MyDrive/noisy_indices.csv", delimiter=",")).astype(int)
indices = indices.tolist()

# convert eagertensor to numpy
# y_train_numpy = [value.numpy() for value in y_train]

# create a x_batch of 1000 values using the 6k noisy examples
noisy_values_y = [y_train[i] for i in indices]
noisy_sample_y = noisy_values_y[:6000:6]

noisy_values_x = [x_train[i] for i in indices]
noisy_sample_x = noisy_values_x[:6000:6]

In [15]:
# get clean indices
total_indices = list(range(60000))
clean_indices = [value for value in total_indices if value not in indices]

# create a sample of 1000 values using the remaining 54k clean examples
clean_values_y = [y_train[i] for i in clean_indices]
clean_sample_y = clean_values_y[:54000:54]

clean_values_x = [x_train[i] for i in clean_indices]
clean_sample_x = clean_values_x[:54000:54]

In [16]:
noisy_sample_x = tf.convert_to_tensor(noisy_sample_x)
noisy_sample_y = tf.convert_to_tensor(noisy_sample_y)
clean_sample_x = tf.convert_to_tensor(clean_sample_x)
clean_sample_y = tf.convert_to_tensor(clean_sample_y)

In [18]:
print("\t","noisy_sample","\t", "clean_sample", "\t","type")
print("x shape" , noisy_sample_x[0].shape,"\t",clean_sample_x[0].shape,"\t", type(noisy_sample_x[0]))
print("y shape" , noisy_sample_y[0].shape,"\t","\t",clean_sample_y[0].shape, "\t","\t",type(clean_sample_y[0]))

	 noisy_sample 	 clean_sample 	 type
x shape (28, 28, 1) 	 (28, 28, 1) 	 <class 'tensorflow.python.framework.ops.EagerTensor'>
y shape (10,) 	 	 (10,) 	 	 <class 'tensorflow.python.framework.ops.EagerTensor'>
total <class 'tensorflow.python.framework.ops.EagerTensor'> 1000
total <class 'tensorflow.python.framework.ops.EagerTensor'> 1000
total <class 'tensorflow.python.framework.ops.EagerTensor'> 1000
total <class 'tensorflow.python.framework.ops.EagerTensor'> 1000


Get gradients for each element: &#8711;$_l (loss(F_d(x_i), y_i))$

In [19]:

batch_size = 32

# comment out as needed
sample_x = noisy_sample_x
sample_y = noisy_sample_y

# sample_x = clean_sample_x
# sample_y = clean_sample_y


individual_gradients = []
for batch_start in range(1000):
    x_batch = sample_x[batch_start:batch_start+batch_size]
    y_batch = sample_y[batch_start:batch_start+batch_size]

    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        pred = model(x_batch)
        loss = cce(pred, y_batch)
        # loss_val = tf.reduce_mean(loss)

    indv_grad = tape.jacobian(loss, model.trainable_variables)
    individual_gradients.append(indv_grad)

Get gradients of batches: &#8711;$_l L(S,F_d))$

In [ ]:
batch_size = 32
batch_gradients = []

for batch_start in range(0, 1000, batch_size):
    x_batch = sample_x[batch_start:batch_start+batch_size]
    y_batch = sample_y[batch_start:batch_start+batch_size]

    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        pred = model(x_batch)
        loss = cce(pred, y_batch)
        loss_val = tf.reduce_mean(loss)

    batch_grad = tape.gradient(loss_val, model.trainable_variables)
    batch_gradients.append(batch_grad)

In [ ]:
import pickle
# comment out as needed

# file_path = '/content/drive/MyDrive/individual_gradients_clean.pkl'
# with open(file_path, 'wb') as file:
#     pickle.dump(individual_gradients, file)
    
file_path = '/content/drive/MyDrive/individual_gradients_noisy.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(individual_gradients, file)

Get a list of neurons for each example

In [ ]:
example_to_neuron = []
for batch_idx, batch in enumerate(batch_gradients):
    print("Batch Index:", batch_idx)
    for ex_idx in range(batch_idx * 32, (batch_idx + 1) * 32):

      if ex_idx >= 1000:
        break
      
      # compute derivatives of (small l(F_d(x_i), y_i) - L(S,F_d))
      difference = [tf.subtract(t1, t2) for t1, t2 in zip(individual_gradients[ex_idx], batch)]

      neuron_of_layer = []
      for layer_idx, layer in enumerate(difference):
        norms = []

        for neuron_idx in range(layer.shape[-1]):

          if layer_idx in {0, 2, 4, 6}:
            norm = tf.norm(layer[:, :, :, neuron_idx])
            norms.append(norm)
          elif layer_idx in {8, 10}:
            norm = tf.norm(layer[:, neuron_idx])
            norms.append(norm)
          else:
            continue
        # print("layer index:", layer_idx, "len of norms:", len(norms))

        if layer_idx in {1,3,5,7,9,11,12,13}:
          continue
        # Extract the index with the highest norm
        imp_norm = tf.argmax(norms)
        neuron_of_layer.append(imp_norm)

      example_to_neuron.append(neuron_of_layer)

# Access the result as needed
print("Example to Neuron Mapping:")
print(example_to_neuron)

In [ ]:
import pickle
# comment as needed

# file_path = '/content/drive/MyDrive/examples_to_neurons_clean.pkl'
# with open(file_path, 'wb') as file:
#     pickle.dump(example_to_neuron, file)
    
file_path = '/content/drive/MyDrive/examples_to_neurons_noisy.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(example_to_neuron, file)
    
